## Decorators

The question I kept repeating while going through so many decorator guides and blogs was why do we need them. Basically decorators allow you to write modular code by making it possible to add or wrap functionality to/around existing functions without changing them.
OK! but WHY do we need the decorator syntax? Why do we need nested functions to create a decorator? Why can't we just take an existing function, write another one with some extra functionality, and feed the first to the 2nd function to make it do something else?

Source: https://stackoverflow.com/questions/52593649/what-is-the-purpose-of-decorators-why-use-them

The current method of applying a transformation to a function or method places the actual transformation after the function body. For large functions this separates a key component of the function's behavior from the definition of the rest of the function's external interface. 
This becomes less readable with longer methods. It also seems less than pythonic to name the function three times for what is conceptually a single declaration. A solution to this problem is to move the transformation of the method closer to the method's own declaration. The intent of the new syntax is to replace

```python
def foo(cls):
    pass 
foo = synchronized(lock)(foo) 
foo = classmethod(foo)
```

with

```python
@classmethod
@synchronized(lock)
def foo(cls):
    pass
```

### Python functions are first class objects
What it means is that we can assign them to variables, store them in data structures, pass them as arguments to other functions, and also return them as values from other functions.

In [1]:
def add_two(x):
    return x+2

# function assigned to a variable and called via the variable name
add = add_two
add(4)

6

In [2]:
add_two(4)

6

So when a function is mentioned with just its name without `()` it is passed as a reference to the expression or variable and when `()` are added it becomes a call to the function.

### Inner functions in Python

In [4]:
def f():
    print("This is pre-definition of 'g'")
    def g():
        print("Hi, it's me 'g'")
        print("Thanks for calling me")
        
    print("This is the function 'f'")
    print("I am calling 'g' now:")
    g()

    
f()

This is pre-definition of 'g'
This is the function 'f'
I am calling 'g' now:
Hi, it's me 'g'
Thanks for calling me


See how the statements in the function `g` are only executed when g is called and not during the definition of it is made in the function `f`. 

Below is a case with arguments in the function call and a return statement as well.

In [7]:
def say_hi(to_whom:str):
    def say_lovely_hi_to(name: str):
        return name + "! "+ "What a lovely day!"
    
    return "Hi "+ say_lovely_hi_to(to_whom)

say_hi('Jane')


'Hi Jane! What a lovely day!'

A useful example of inner function architecture, here's a function that calculates factorial of a number and also checks the mentioned conditions in the start. However the one issue is that since the factorial function is being called recursively, it makes those checks on each call which is not relevant if the number passes those checks right at the first time.

In [8]:
def factorial(n):
    """ calculates the factorial of n, 
        n should be an integer and n <= 0 """
    if type(n) == int and n >=0:
        if n == 0:
            return 1
        else:
            return n * factorial(n-1)

    else:
        raise TypeError("n has to be a positive integer or zero")

Here's the inner function way to do it, which makes the check only once and then proceeds with the factorial calculation.

In [10]:
def factorial(n):
    """ calculates the factorial of n, if n is either a non negative
    integer or a float number x being equivalent to an integer, like
    4.0, 12.0, 8. i.e. no decimals following the decimal point """
    def inner_factorial(n):
        if n == 0:
            return 1
        else:
            return n * inner_factorial(n-1)
    if not isinstance(n, (int, float)):
        raise ValueError("Value is neither an integer nor a float equivalent to int")
    if isinstance(n, (int))  and n < 0:
        raise ValueError('Should be a positive integer or 0')
    elif isinstance(n, (float)) and not n.is_integer():
        raise ValueError('value is a float but not equivalent to an int')
    else:
        return inner_factorial(n)
    

values = [0, 1, 8, 9.0, -2, 5.3, "5"]
for value in values:
    try: 
        print(value, end=", ")
        print(factorial(value))
    except ValueError as e:
        print(e)

0, 1
1, 1
8, 40320
9.0, 362880.0
-2, Should be a positive integer or 0
5.3, value is a float but not equivalent to an int
5, Value is neither an integer nor a float equivalent to int


### Function as a parameter
Now that we know that the functions are first class objects in Python, let's see how that goes

In [11]:
def g():
    print("Hi, it's me 'g'")
    print("Thanks for calling me")
    
def f(func):
    print("Hi, it's me 'f'")
    print("I will call 'func' now")
    func()
          
f(g)

Hi, it's me 'f'
I will call 'func' now
Hi, it's me 'g'
Thanks for calling me


Here's another one like that

In [13]:
import math

def foo(func):
    print("The function " + func.__name__ + " was passed to foo")
    return func(1)

print(foo(math.sin))
print(foo(math.cos))

The function sin was passed to foo
0.8414709848078965
The function cos was passed to foo
0.5403023058681398


This one below is a super useful example where the goal is to time the execution of a function passed as an argument.

In [26]:
import time
def timer(func):
    """Print the runtime of the function"""
    start_time = time.perf_counter()
    value = func()
    end_time = time.perf_counter()
    run_time = end_time - start_time
    print(f"Finished in {run_time:.4f} secs")

def calc_sin():
    for i in range(0,10000):
        math.sin(i) 

timer(calc_sin)

Finished in 0.0011 secs


## Enter Decorators

In [12]:
import time
import math

def timer_decorator(func):
    print("Decorator function is called right at the '@' point so that the inner function is returned")
    def actual_timer_function():
        """Print the runtime of the function"""
        print("Now the inner function is called")
        start_time = time.perf_counter()
        func()
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished in {run_time:.4f} secs")
    return actual_timer_function

def calc_sin():
    for i in range(0,10000):
        math.sin(i) 


Instead of writing calc_abc = timer_decorator(calc_sin) we use this other notation.

In [13]:
calc_abc = timer_decorator(calc_sin)


Decorator function is called right at the '@' point so that the inner function is returned


In [14]:
calc_abc()

Now the inner function is called
Finished in 0.0011 secs


In [15]:
@timer_decorator
def calc_sin():
    for i in range(0,10000):
        math.sin(i) 

print("calling the decorated function now (it's the same as passing calc_sin function to timer_decorator to call in the actual_timer_function)")
calc_sin()

Decorator function is called right at the '@' point so that the inner function is returned
calling the decorated function now (it's the same as passing calc_sin function to timer_decorator to call in the actual_timer_function)
Now the inner function is called
Finished in 0.0015 secs


### `__name__` feature

In [16]:
def our_decorator(func):
    def function_wrapper(x):
        print("Before calling " + func.__name__)
        func(x)
        print("After calling " + func.__name__)
    return function_wrapper

@our_decorator
def foo(x):
    print("Hi, foo has been called with " + str(x))

foo("Hi")

Before calling foo
Hi, foo has been called with Hi
After calling foo
